In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.colors
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.figure_factory as ff

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [ ]:
### Load Data Files
## Train Data
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")

## Test Data
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")

## Stores Data
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")

## Oil Data 
oils = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")

## Holidays
holidays = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")

## Transactions
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv")


In [ ]:
train.head

train.shape

In [ ]:
stores.head()
## number of product families in a year
print("Number of stores : {} \n".format(train.store_nbr.nunique()))

In [ ]:
oils.head()

print("The oils data begins on {} and ends on {}.\n".format(oils.date.min(),oils.date.max()))
print("Number of days with oil prices : {} .\n".format(oils.date.nunique()))


### Check for duplicates in python

duplicate_dates = oils["date"].duplicated(keep = 'first').nunique()

duplicate_dates

In [ ]:
transactions.head()

In [ ]:
print("The transactions data begins on {} and ends on {}.\n".format(transactions.date.min(),transactions.date.max()))
print("Number of days with transactions : {} .\n".format(transactions.date.nunique()))
print("Number of stores with transactions data in every {} \n".format(transactions.groupby(pd.to_datetime(train['date'])).store_nbr.nunique()))

In [ ]:
## number of product families in a year
print("The training data has the following number of family products in every year {} \n".format(train.groupby(pd.to_datetime(train['date']).dt.year).family.nunique()))

In [ ]:
### Number of stores in every year
print("The training data has the following number of stores in every year {} \n".format(train.groupby(pd.to_datetime(train['date']).dt.year).store_nbr.nunique()))

In [ ]:
print("The training data begins on {} and ends on {}.\n".format(train.date.min(),train.date.max()))
print("Number of days with transactions : {} .\n".format(train.date.nunique()))
print("The test data begins on {} and ends on {}.\n".format(test.date.min(),test.date.max()))



In [ ]:
def summarize(df,  n_rows_to_show=5):
    """Simply summarize the given DataFrame.

    Parameters:
    df: pd.DataFrame, raw DataFrame
    file_name: str, name of the file
    n_rows_to_show: int, number of rows to show
    """
    print(f"Shape: {df.shape}")

    nan_ratio = pd.isna(df).sum() / len(df) * 100
    nan_ratio.sort_values(ascending=False, inplace=True)
    nan_ratio = nan_ratio.to_frame(name='NaN Ratio').T
    print("NaN ratio:")
    display(nan_ratio)

    display(df.head(n_rows_to_show))

In [ ]:
summarize(train, 5)
summarize(test, 5)

### Merging the datasets

In [ ]:
## Merge Store Details
training_data = train.merge(stores , how = 'left', on = ['store_nbr'])

## Merge Number of Transactions
training_data = training_data.merge(transactions , how = 'left', on = ['date', 'store_nbr'])

## Merge Number of Transactions
training_data = training_data.merge(oils , how = 'left', on = ['date'])

## Add day of the month 
training_data['Month'] = pd.to_datetime(training_data['date']).dt.month

## Add day of the week
training_data['day_of_week'] = pd.to_datetime(training_data['date']).dt.day_name()

## Add Year
training_data['year'] = pd.to_datetime(training_data['date']).dt.year

## Summarize merged dataset

In [ ]:
### Check missing values
summarize(training_data, 5)

### Check for duplicates
duplicate_dates = oils["date"].duplicated(keep = 'first').nunique()
print("Number of days with uplicates stores {}.\n.format(duplicate_dates)")

In [ ]:
#### Find the days when there are most sales and transactions
### How the price of oil affect the voulme of sales and number of transactions
### Find the two days where there are no transactions recorded
## Number of days with no oil prices
## Determine the family of products under promotion Top 10 and Bottom 10
## Average Number of sales per month, compared to the average sales in every holiday

In [ ]:
df1 = training_data.loc[training_data["year"] == 2017]

## Average
df = df1.groupby('date').mean()['sales']

fig = px.line(df, y="sales" )
fig.show()




In [ ]:
df = df1.groupby('date').mean()['dcoilwtico']
fig = px.line(df, y="dcoilwtico" )
fig.show()

In [ ]:
colors=px.colors.qualitative.Plotly
temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), width=1400))

In [ ]:
train_date=training_data.date.unique()
total_sales=training_data.groupby('date')['sales'].sum().div(100)
promotion=training_data.groupby('date')['onpromotion'].sum()
avg_sales=training_data.groupby('date')['sales'].mean()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=train_date,
        y= total_sales,
        name = "total sales", mode='lines'
    ))

fig.add_trace(
    go.Scatter(
        x=train_date,
        y= promotion,
        name = "promotionL", mode='lines'
    ))

# fig.add_trace(
#     go.Bar(
#         x=train_date,
#         y=promotion,
#         name = "promotion"
#     ))


fig.update_xaxes(rangeslider_visible= True,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=2, label="2y", step="year", stepmode="backward"),
                         dict(count=4, label="4y", step="year", stepmode="backward"),
                         dict(step="all")])))
fig.update_layout(template=temp,title='Total Sales/100  and Items on Promotion', 
                  autosize=False, 
                  width=1400, 
                  height=700, 
                  xaxis_title="Date", 
                  yaxis_title="",
                  hovermode='x unified',
                  showlegend=True)
fig.show()


### Average Sales with the price of oil

In [ ]:
### drop days without oil prices

training_data1 = training_data.copy()
print(training_data1.shape)

training_data1 = training_data1.dropna(subset=['dcoilwtico'])
print(training_data1.shape)

transactions_num=training_data1.groupby(['date', 'store_nbr'])['transactions'].first()

transactions_num.columns = [
    'date', 
    'store_nbr',
    'transactions']
transactions_num = transactions_num.reset_index()

transactions_num = transactions_num.groupby('date')['transactions'].sum().div(1000)


train_date=training_data1.date.unique()


train_date1 =pd.DataFrame(train_date, columns = ['date'])

train_date1= train_date1.merge(day, on = ['date'])

train_date1['date_day'] = train_date1['date'] + " " + train_date1['day_of_week']


oil_prices=training_data1.groupby('date')['dcoilwtico'].first()

sales=training_data1.groupby('date')['sales'].sum().div(20000)

day = training_data1.groupby('date')['day_of_week'].first()



fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x= train_date1['date_day'], ##[train_date, day],
        y= oil_prices,
        name = 'Oil Prices',
        mode='lines'
    ))

fig.add_trace(
    go.Scatter(
        x= train_date1['date_day'], ## [train_date, day],
        y= sales,
        name = "Sales/(10000)",
        mode='lines'
    ))

fig.add_trace(
    go.Scatter(
        x= train_date1['date_day'], ##[train_date, day],
        y= transactions_num,
        name = "Number of Transactions", mode='lines'
    ))

fig.update_xaxes(rangeslider_visible= True,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=2, label="2y", step="year", stepmode="backward"),
                         dict(count=4, label="4y", step="year", stepmode="backward"),
                         dict(step="all")])))
fig.update_layout(
    
    
    
    
    
    
    
    
    
    late=temp,title='Sales/10000  and Oil prices and Number of Transactions', 
                  autosize=False, 
                  width=1400, 
                  height=700, 
                  xaxis_title="Date", 
                  yaxis_title="",
                  hovermode='x unified',
                  showlegend=True)
fig.show()


## Oil Prices and days of the week

In [ ]:
oil_days = training_data.groupby("day_of_week")['dcoilwtico'].count()
print(oil_days)

In [ ]:
#check = train['date'].value_counts().reset_index()
check=training_data.groupby(['family','year']).agg({"sales" : "mean"}).reset_index()

check.columns = [
    'family', 
    'year',
    'sales'
]

check = check.sort_values('sales', ascending=False)

fig = px.histogram(check, x="family", y="sales",
             color='year', barmode='group',
             height=400)
fig.show()

In [ ]:
#check = train['date'].value_counts().reset_index()
check=training_data.groupby(['family','year']).agg({"sales" : "sum"}).reset_index()

check.columns = [
    'family', 
    'year',
    'sales'
]

check = check.sort_values('sales', ascending=False)

fig = px.histogram(check, x="family", y="sales",
             color='year', barmode='group',
             height=400)
fig.show()

In [ ]:

training_data =training_data.sort_values('sales', ascending=False)

fig = px.box(training_data, x="family", y="sales",  color='year',
             height=400)
fig.show()